# Punto # 1
En este primer punto, vamos a **crear un DataFrame** que contenga 1000 registros, donde cada registro representa un producto con ciertas características:
- **Producto**: formato `P0001`, `P0002`, ..., `P1000`
- **Ventas**: un valor aleatorio entre 1000 y 10000
- **Categoría**: depende de si el número de producto es par (GPAR) o impar (GIMPAR)

Además, vamos a **calcular una nueva columna** llamada **"Venta Neta"**, que se obtiene restando el IGV (18% de las Ventas) y un **descuento** que varía según el rango de la venta. Esto nos va a permitir tener un indicador final de la venta después de impuestos y descuentos.

### Carga de librerías
Para empezar, importamos las librerías que utilizaremos para manejar y analizar datos. Principalmente, usaremos **pandas** y **numpy**.

In [30]:
import pandas as pd  # Librería para manipulación y análisis de datos
import numpy as np  # Librería para operaciones numéricas y generación de datos aleatorios
import os  # Librería para interactuar con el sistema de archivos

### Funciones para la generación y el cálculo
En este bloque, definiremos un par de funciones:
1. **`generar_dataframe()`**: creará el DataFrame con las columnas `Producto`, `Ventas` y `Categoría`, siguiendo todos los criterios solicitados.
2. **`calcular_venta_neta()`**: a partir del DataFrame ya creado, calculará la columna adicional `Venta Neta` restando el IGV y el descuento correspondiente.

In [31]:
# Definir función para generar el DataFrame con las condiciones establecidas
def generar_dataframe():
    # Establecer la semilla para la reproducibilidad
    np.random.seed(42)
    
    # Crear lista de productos con formato P0001, P0002, ..., P1000
    productos = [f'P{i:04d}' for i in range(1, 1001)]
    
    # Generar valores aleatorios de ventas entre 1000 y 10000
    ventas = np.random.randint(1000, 10001, size=1000)
    
    # Asignar categorías según la paridad del número del producto
    categorias = ['GIMPAR' if int(p[1:]) % 2 != 0 else 'GPAR' for p in productos]
    
    # Crear DataFrame inicial
    df = pd.DataFrame({
        'Producto': productos,
        'Ventas': ventas.astype(float),  # Asegurar tipo float
        'Categoría': pd.Categorical(categorias)
    })
    return df

Ahora, la función para calcular la **Venta Neta** tendrá en cuenta:
- **IGV** de 18% sobre el valor de `Ventas`
- **Descuento** variable de acuerdo a los rangos de la tabla proporcionada.

Al final, crearemos una nueva columna en el DataFrame denominada `Venta Neta` y eliminaremos las columnas intermedias de IGV y Descuento para no saturar la tabla.

In [32]:
# Definir función para calcular Venta Neta
def calcular_venta_neta(df):
    # Calcular el IGV (18% de las ventas)
    df['IGV'] = df['Ventas'] * 0.18

    # Definir función para calcular descuento según el rango de ventas
    def calcular_descuento(venta):
        if 1000 <= venta < 1500:
            return venta * 0.01  # 1%
        elif 1500 <= venta < 3000:
            return venta * 0.02  # 2%
        elif 3000 <= venta < 7000:
            return venta * 0.03  # 3%
        elif 7000 <= venta < 9000:
            return venta * 0.04  # 4%
        else:
            return venta * 0.05  # 5%
    
    # Aplicar la función de descuento
    df['Descuento'] = df['Ventas'].apply(calcular_descuento)
    
    # Calcular la Venta Neta
    df['Venta Neta'] = df['Ventas'] - df['IGV'] - df['Descuento']
    
    # Eliminar las columnas intermedias si no son necesarias
    df.drop(columns=['IGV', 'Descuento'], inplace=True)
    return df

### Aplicación de las funciones
Procedemos a generar el DataFrame con `generar_dataframe()` y, a continuación, usamos `calcular_venta_neta()` para añadir la columna que nos interesa. Luego mostramos los primeros 5 registros a modo de ejemplo.

In [33]:
df = generar_dataframe()
df = calcular_venta_neta(df)
df.head(5)

,Producto,Ventas,Categoría,Venta Neta
0,P0001,8270.0,GIMPAR,6450.60
1,P0002,1860.0,GPAR,1488.00
2,P0003,6390.0,GIMPAR,5048.10
3,P0004,6191.0,GPAR,4890.89
4,P0005,6734.0,GIMPAR,5319.86


Para tener un respaldo de este DataFrame, lo exportamos también a un archivo CSV, por si más adelante se necesita revisar o compartir la información.

In [35]:
# Exportar el DataFrame a un archivo CSV
df.to_csv('ventas_netas.csv', index=False, encoding='utf-8')
print("El DataFrame ha sido guardado en 'ventas_netas.csv' exitosamente.")

El DataFrame ha sido guardado en 'ventas_netas.csv' exitosamente.


# Punto # 2
En este segundo punto, se pide exportar el **dataframe inicial** en formato **Parquet** y usar compresión `Snappy`. Después, se deberá **importar a Power BI**, donde se creará la columna `Venta Neta` (mediante DAX) y se realizarán visualizaciones.
Si bien aquí no vamos a visualizar en Power BI (eso se hace dentro de la propia herramienta), sí mostraremos el paso de guardar en Parquet y el ejemplo de DAX que podría utilizarse.

In [28]:
# Guardar el DataFrame resultante en un archivo Parquet con compresión Snappy
df = generar_dataframe() # Generar el DataFrame original
df.to_parquet('ventas_netas.parquet', index=False, compression='snappy')
print("El DataFrame ha sido guardado en 'ventas_netas.parquet' con compresión Snappy.")

El DataFrame ha sido guardado en 'ventas_netas.parquet' con compresión Snappy.


In [61]:
with open("venta_neta.dax", "w") as f:
    f.write("""
Venta Neta = 
VAR VentasActuales = 'ventas_netas'[Ventas]
VAR IGV = VentasActuales * 0.18
VAR DescuentoPorc =
    SWITCH(
        TRUE(),
        VentasActuales <= 1500, 0.01,
        VentasActuales <= 3000, 0.02,
        VentasActuales <= 7000, 0.03,
        VentasActuales <= 9000, 0.04,
        0.05
    )
VAR Descuento = VentasActuales * DescuentoPorc
RETURN
    VentasActuales - IGV - Descuento
""")

El archivo `venta_neta.dax` contiene una medida de ejemplo en DAX para que, al cargar el archivo Parquet a Power BI, se pueda **crear la misma columna** `Venta Neta` de manera dinámica. Luego, en Power BI, se pueden crear gráficos de barras, líneas o tablas dinámicas para analizar las ventas.

# Punto # 3 (de mayor importancia)
En este tercer punto, trabajaremos con un **Libro de Excel** llamado `BASES_MODELO.xlsx`, que contiene varias hojas:
- **BASE_VARIABLES**: con información detallada de ventas y productos.
- **BASE_ESTIMADOS**: con las estimaciones de mercadeo.
- **BASE_ASESORAS**: información sobre la cantidad de asesoras por zona y campaña.

La tarea consiste en:
1. **Unir la información** de las tres hojas en un solo DataFrame.
2. **Construir un modelo predictivo** (nuestra variable objetivo es `VENTA`).
3. **Comparar el poder predictivo** del modelo vs. las estimaciones de mercadeo (`CANTIDADES_ESTIMADAS`).

Vamos a realizar todos estos pasos en Python, mostrando las uniones, la limpieza y el entrenamiento de un par de modelos (Regresión Lineal y Random Forest). Por último, compararemos su desempeño con las estimaciones que se tenían originalmente.

In [44]:
# ------------------------------------------------------------------------------
# 1. IMPORTACIÓN DE LIBRERÍAS Y LECTURA DE EXCEL
# ------------------------------------------------------------------------------

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Leemos las hojas del archivo BASES_MODELO
ruta_excel = "BASES_MODELO.xlsx"

df_variables = pd.read_excel(ruta_excel, sheet_name="BASE_VARIABLES")
df_estimados = pd.read_excel(ruta_excel, sheet_name="BASE_ESTIMADOS")
df_asesoras = pd.read_excel(ruta_excel, sheet_name="BASE_ASESORA")

print("=== BASE_VARIABLES ===")
display(df_variables.head())

print("\n=== BASE_ESTIMADOS ===")
display(df_estimados.head())

print("\n=== BASE_ASESORAS ===")
display(df_asesoras.head())

=== BASE_VARIABLES ===


,CAMPANA,CODIGO_DEL_PRODUCTO,VENTA,VENTA_ZONA_101,VENTA_ZONA_102,VENTA_ZONA_103,VENTA_ZONA_104,VENTA_ZONA_107,VENTA_ZONA_109,VENTA_ZONA_110,VENTA_ZONA_111,VENTA_ZONA_112,VENTA_ZONA_115,VENTA_ZONA_116,VENTA_ZONA_119,NOMB_SUBGRUPO,TALLA,PRECIO_NAC
0,201902,P00845,25,0,0,0,0,0,0,0,0,0,0,0,0,201 RE-BLUSAS FEM,T-XL,99900
1,201908,P03495,25,0,0,0,0,0,0,0,0,0,0,0,0,201 RE-BLUSAS FEM,T-L,99900
2,201910,P04897,25,0,0,0,0,0,0,0,0,0,0,0,0,201 RE-BLUSAS FEM,T-L,99900
3,201907,P03019,25,0,0,0,0,0,0,0,0,0,0,0,0,215 RE-FALDAS FEM,T-XS,99900
4,201903,P01075,25,0,0,0,0,0,0,0,0,0,0,0,0,216 RE-JEANS FEM,T-8,99900



=== BASE_ESTIMADOS ===


,CAMPANA,CODIGO_DEL_PRODUCTO,CANTIDADES_ESTIMADAS
0,201901,P00001,3562
1,201901,P00002,521
2,201901,P00003,4053
3,201901,P00004,16398
4,201901,P00005,2584



=== BASE_ASESORAS ===


,CAMPANA,N° ASESORAS_ZONA_101,N° ASESORAS_ZONA_102,N° ASESORAS_ZONA_103,N° ASESORAS_ZONA_104,N° ASESORAS_ZONA_107,N° ASESORAS_ZONA_109,N° ASESORAS_ZONA_110,N° ASESORAS_ZONA_111,N° ASESORAS_ZONA_112,N° ASESORAS_ZONA_115,N° ASESORAS_ZONA_116,N° ASESORAS_ZONA_119,N° ASESORAS
0,201901,903,679,554,289,542,483,421,521,514,431,675,419,24133
1,201902,802,654,512,277,511,449,455,434,457,376,556,389,22840
2,201903,896,700,525,281,565,467,399,491,498,437,587,402,23832
3,201904,821,670,537,284,573,452,398,429,459,412,595,385,23243
4,201905,801,637,523,271,424,442,306,454,419,420,603,369,22874


### Unión de datos
Para combinar la información:
1. **Unimos** `BASE_VARIABLES` con `BASE_ESTIMADOS` por las columnas en común (`CAMPANA` y `CODIGO_DEL_PRODUCTO`), añadiendo la columna `CANTIDADES_ESTIMADAS`.
2. **Derretimos** (melt) el DataFrame de `BASE_ASESORAS` para obtener tres columnas principales: `CAMPANA`, `ZONA` y `ASESORAS`.
3. **Unimos** esa información de asesoras con el DataFrame anterior (también por la columna `CAMPANA`). Esto nos permite después, por cada zona, tener el número de asesoras.

Al final, tendremos un `df_final` que consolida toda la información que necesitamos para el modelado.

In [46]:
# ------------------------------------------------------------------------------
# 2. UNIÓN DE DATAFRAMES
# ------------------------------------------------------------------------------

# Unimos BASE_VARIABLES con BASE_ESTIMADOS usando 'CODIGO_DEL_PRODUCTO' y 'CAMPANA'
df_merged_1 = pd.merge(
    df_variables,
    df_estimados[['CAMPANA', 'CODIGO_DEL_PRODUCTO', 'CANTIDADES_ESTIMADAS']], 
    on=['CAMPANA', 'CODIGO_DEL_PRODUCTO'],
    how='left'  # 'left' para conservar filas de df_variables
)

# Reestructuramos df_asesoras para que tenga columnas CAMPANA, ZONA, y ASESORAS
df_asesoras_melted = df_asesoras.melt(id_vars=['CAMPANA'], 
                                      var_name='ZONA', 
                                      value_name='ASESORAS')

# Limpiamos los nombres de zona (eliminamos 'N° ASESORAS_ZONA_')
df_asesoras_melted['ZONA'] = df_asesoras_melted['ZONA'].str.replace('N° ASESORAS_ZONA_', '')

# Agregamos la información de BASE_ASESORAS
df_final = pd.merge(
    df_merged_1,
    df_asesoras_melted,
    on=['CAMPANA'],
    how='left'
)

print("Dimensiones del dataframe final:", df_final.shape)
display(df_final.head(10))

Dimensiones del dataframe final: (75621, 21)


,CAMPANA,CODIGO_DEL_PRODUCTO,VENTA,VENTA_ZONA_101,VENTA_ZONA_102,VENTA_ZONA_103,VENTA_ZONA_104,VENTA_ZONA_107,VENTA_ZONA_109,VENTA_ZONA_110,...,VENTA_ZONA_112,VENTA_ZONA_115,VENTA_ZONA_116,VENTA_ZONA_119,NOMB_SUBGRUPO,TALLA,PRECIO_NAC,CANTIDADES_ESTIMADAS,ZONA,ASESORAS
0,201902,P00845,25,0,0,0,0,0,0,0,...,0,0,0,0,201 RE-BLUSAS FEM,T-XL,99900,46,101,802
1,201902,P00845,25,0,0,0,0,0,0,0,...,0,0,0,0,201 RE-BLUSAS FEM,T-XL,99900,46,102,654
2,201902,P00845,25,0,0,0,0,0,0,0,...,0,0,0,0,201 RE-BLUSAS FEM,T-XL,99900,46,103,512
3,201902,P00845,25,0,0,0,0,0,0,0,...,0,0,0,0,201 RE-BLUSAS FEM,T-XL,99900,46,104,277
4,201902,P00845,25,0,0,0,0,0,0,0,...,0,0,0,0,201 RE-BLUSAS FEM,T-XL,99900,46,107,511


### Limpieza y verificación
En esta fase, comprobamos si existen datos faltantes (nulos) en las nuevas columnas creadas. De ser necesario, podríamos imputarlos con ceros u otro método. También verificamos los tipos de datos y si hay posibles valores atípicos (outliers).

In [47]:
# ------------------------------------------------------------------------------
# 2.1. LIMPIEZA BÁSICA DE DATOS
# ------------------------------------------------------------------------------
print("Valores Nulos por columna:\n", df_final.isnull().sum())
df_final['CANTIDADES_ESTIMADAS'] = df_final['CANTIDADES_ESTIMADAS'].fillna(0)
df_final['ASESORAS'] = df_final['ASESORAS'].fillna(0)


Valores Nulos por columna:
 CAMPANA                 0
CODIGO_DEL_PRODUCTO     0
VENTA                   0
VENTA_ZONA_101          0
VENTA_ZONA_102          0
VENTA_ZONA_103          0
VENTA_ZONA_104          0
VENTA_ZONA_107          0
VENTA_ZONA_109          0
VENTA_ZONA_110          0
VENTA_ZONA_111          0
VENTA_ZONA_112          0
VENTA_ZONA_115          0
VENTA_ZONA_116          0
VENTA_ZONA_119          0
NOMB_SUBGRUPO           0
TALLA                   0
PRECIO_NAC              0
CANTIDADES_ESTIMADAS    0
ZONA                    0
ASESORAS                0
dtype: int64


In [48]:
# ------------------------------------------------------------------------------
# 3. ANÁLISIS EXPLORATORIO (Opcional / Resumido)
# ------------------------------------------------------------------------------
print(df_final.describe())

# Matriz de correlaciones para ver relaciones generales
corr_matrix = df_final.corr(numeric_only=True)
print("\nMatriz de correlación:\n", corr_matrix)

             CAMPANA         VENTA  VENTA_ZONA_101  VENTA_ZONA_102  \
count   75621.000000  75621.000000    75621.000000    75621.000000   
mean   201906.420148   3590.018566      130.017019      104.457796   
std         3.308503   3332.248149      185.567726      136.119998   
min    201901.000000     25.000000        0.000000        0.000000   
25%    201904.000000   1428.000000        0.000000        0.000000   
50%    201906.000000   2639.000000       76.000000       64.000000   
75%    201909.000000   4620.000000      178.000000      152.000000   
max    201912.000000  34645.000000     4001.000000     1588.000000   

       VENTA_ZONA_103  VENTA_ZONA_104  VENTA_ZONA_107  VENTA_ZONA_109  \
count    75621.000000    75621.000000    75621.000000    75621.000000   
mean        85.258037       41.903559       82.692969       73.810727   
std        169.415072       68.874934      152.478573      132.099978   
min          0.000000        0.000000        0.000000        0.000000   
25% 

### Entrenamiento y validación
Finalmente, vamos a entrenar dos modelos sencillos (una Regresión Lineal y un Random Forest) para **predecir** la variable objetivo `VENTA`. Dividiremos los datos en entrenamiento y prueba basándonos en la campaña: tomamos **todas las campañas** anteriores como entrenamiento y la **última campaña** como conjunto de prueba.
Esto simula un escenario real donde se tiene información histórica y se entrena el modelo para pronosticar la campaña más reciente.

In [51]:
# ------------------------------------------------------------------------------
# 4. MODELADO PREDICTIVO
# ------------------------------------------------------------------------------

# 4.1. Definimos la variable respuesta (y) y las variables explicativas (X)
# ------------------------------------------------------------------------------
df_model = df_final.copy()

# Convertimos 'ZONA' en dummies para poder usarlo como variable en el modelo (opcional)
df_model = pd.get_dummies(df_model, columns=["ZONA"], prefix="ZONA", drop_first=True)

# Separamos la variable objetivo (VENTA)
X = df_model.drop(columns=["VENTA", "CODIGO_DEL_PRODUCTO", "CAMPANA"])
y = df_model["VENTA"]

# 4.2. Dividimos datos en entrenamiento y validación (campaña)
campaña_max = df_model["CAMPANA"].max()
print("Última campaña en los datos:", campaña_max)

mascara_train = df_model["CAMPANA"] < campaña_max
mascara_test  = df_model["CAMPANA"] == campaña_max

X_train = X[mascara_train].drop(columns=["CAMPANA"], errors="ignore")
y_train = y[mascara_train]

X_test = X[mascara_test].drop(columns=["CAMPANA"], errors="ignore")
y_test = y[mascara_test]

print("Tamaño X_train:", X_train.shape)
print("Tamaño X_test:", X_test.shape)

Última campaña en los datos: 201912
Tamaño X_train: (72111, 29)
Tamaño X_test: (3510, 29)


Aquí simplemente estamos dejando fuera del conjunto de entrenamiento la campaña más reciente para poder probar el modelo con datos que el modelo **no ha visto**. Esto da una idea más realista de la capacidad de generalización.

In [58]:
# ------------------------------------------------------------------------------
# 4.3 ENTRENAMIENTO DE MODELOS
# ------------------------------------------------------------------------------
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Eliminamos columnas categóricas que no estén transformadas
columnas_a_eliminar = ["NOMB_SUBGRUPO", "TALLA"]
X_train = X_train.drop(columns=columnas_a_eliminar, errors="ignore").select_dtypes(include=['number'])
X_test = X_test.drop(columns=columnas_a_eliminar, errors="ignore").select_dtypes(include=['number'])

print("Columnas en X_train después de la limpieza:", X_train.columns.tolist())
print("Columnas en X_test después de la limpieza:", X_test.columns.tolist())

# MODELO 1: REGRESIÓN LINEAL
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_pred_linreg = linreg.predict(X_test)
mae_linreg = mean_absolute_error(y_test, y_pred_linreg)
rmse_linreg = mean_squared_error(y_test, y_pred_linreg) ** 0.5
print("\n=== Regresión Lineal ===")
print(f"MAE: {mae_linreg:.2f}")
print(f"RMSE: {rmse_linreg:.2f}")

# MODELO 2: RANDOM FOREST
rforest = RandomForestRegressor(n_estimators=100, random_state=42)
rforest.fit(X_train, y_train)
y_pred_rf = rforest.predict(X_test)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = mean_squared_error(y_test, y_pred_rf) ** 0.5
print("\n=== Random Forest ===")
print(f"MAE: {mae_rf:.2f}")
print(f"RMSE: {rmse_rf:.2f}")

Columnas en X_train después de la limpieza: ['VENTA_ZONA_101', 'VENTA_ZONA_102', 'VENTA_ZONA_103', 'VENTA_ZONA_104', 'VENTA_ZONA_107', 'VENTA_ZONA_109', 'VENTA_ZONA_110', 'VENTA_ZONA_111', 'VENTA_ZONA_112', 'VENTA_ZONA_115', 'VENTA_ZONA_116', 'VENTA_ZONA_119', 'PRECIO_NAC', 'CANTIDADES_ESTIMADAS', 'ASESORAS']
Columnas en X_test después de la limpieza: ['VENTA_ZONA_101', 'VENTA_ZONA_102', 'VENTA_ZONA_103', 'VENTA_ZONA_104', 'VENTA_ZONA_107', 'VENTA_ZONA_109', 'VENTA_ZONA_110', 'VENTA_ZONA_111', 'VENTA_ZONA_112', 'VENTA_ZONA_115', 'VENTA_ZONA_116', 'VENTA_ZONA_119', 'PRECIO_NAC', 'CANTIDADES_ESTIMADAS', 'ASESORAS']

=== Regresión Lineal ===
MAE: 583.78
RMSE: 960.60

=== Random Forest ===
MAE: 615.82
RMSE: 967.40


Observamos que la **Regresión Lineal** obtuvo un mejor desempeño que el **Random Forest** en este ejemplo, al menos con la configuración por defecto.
El **MAE** (Error Absoluto Medio) y el **RMSE** (Raíz del Error Cuadrático Medio) son menores en la Regresión Lineal, lo cual sugiere que se ajusta mejor a estos datos específicos.

In [60]:
# ------------------------------------------------------------------------------
# 5. COMPARACIÓN CON LAS CANTIDADES_ESTIMADAS
# ------------------------------------------------------------------------------
df_test = df_final[df_final["CAMPANA"] == campaña_max].copy()

# Tomamos las cantidades estimadas por mercadeo
y_pred_marketing = df_test["CANTIDADES_ESTIMADAS"]
y_real = df_test["VENTA"]

mae_marketing = mean_absolute_error(y_real, y_pred_marketing)
rmse_marketing = mean_squared_error(y_real, y_pred_marketing) ** 0.5

print("\n=== Estimaciones de Mercadeo ===")
print(f"MAE: {mae_marketing:.2f}")
print(f"RMSE: {rmse_marketing:.2f}")


=== Estimaciones de Mercadeo ===
MAE: 2155.94
RMSE: 3686.86


Podemos ver que las **estimaciones de mercadeo** presentan un MAE de 2155.94, muy superior al de nuestros modelos. Es decir, el error medio de las estimaciones de mercadeo es bastante más grande comparado con cualquiera de las dos aproximaciones.

# Conclusiones
1. **Los modelos predictivos superan ampliamente las estimaciones de mercadeo**.
   - Regresión Lineal: MAE ≈ 584, RMSE ≈ 961
   - Random Forest: MAE ≈ 616, RMSE ≈ 967
   - Estimaciones de Mercadeo: MAE ≈ 2156, RMSE ≈ 3687

2. **La Regresión Lineal se comportó mejor que el Random Forest** (al menos con los parámetros predeterminados utilizados en este ejemplo).

3. **Interpretación**: Las estimaciones de mercadeo pueden no estar considerando tantos factores como el modelo, o pueden tener supuestos distintos. Al incorporar más variables (ventas por zonas, número de asesoras, etc.), el modelo consigue un mejor ajuste.

4. **Recomendaciones**:
   - Probar técnicas de optimización de hiperparámetros o modelos más avanzados (XGBoost, LightGBM, etc.).
   - Explorar más variables potencialmente importantes, como datos demográficos, promociones, temporadas, etc.

En definitiva, estos resultados indican que **un enfoque basado en datos y modelos predictivos** puede resultar más preciso que las estimaciones meramente basadas en criterio de mercadeo.